In [3]:
import pandas as pd

# Load the data
df = pd.read_csv('2015_Street_Tree_Census_-_Tree_Data_20241027.csv')

# Display initial shape
print("Initial shape:", df.shape)

# Identify columns with empty values
empty_columns_original = df.columns[df.isnull().any()].tolist()
print("Columns with empty values (original):", empty_columns_original)

# Fill specified columns based on the 'cstatus' column
df.loc[df['status'] == 'Dead', ['health', 'spc_latin', 'spc_common', 'steward', 'guards', 'sidewalk', 'problems']] = 'dead'
df.loc[df['status'] == 'Stump', ['health', 'spc_latin', 'spc_common', 'steward', 'guards', 'sidewalk', 'problems']] = 'stump'
df.loc[df['status'] == 'dead', ['health', 'spc_latin', 'spc_common', 'steward', 'guards', 'sidewalk', 'problems']] = 'dead'
df.loc[df['status'] == 'stump', ['health', 'spc_latin', 'spc_common', 'steward', 'guards', 'sidewalk', 'problems']] = 'stump'


# Identify columns with empty values after filling
empty_columns_after_fill = df.columns[df.isnull().any()].tolist()
#print("Columns with empty values (after fill):", empty_columns_after_fill)

# Drop rows with any empty values in specified key columns
df_cleaned = df.dropna(subset=empty_columns_after_fill)

# Display cleaned shape
print("Cleaned shape:", df_cleaned.shape)
# Save the cleaned data back to Excel
#df_cleaned.to_csv('cleaned_data.csv', index=False
#Used code for making numeric the information below
#print('root stone', df_cleaned['root_stone'].unique())
#print('sidewalk:', df_cleaned['sidewalk'].unique())
#print('guards:', df_cleaned['guards'].unique())
#print('steward:', df_cleaned['steward'].unique())
#print('health:', df_cleaned['health'].unique())
#print('user_type:', df_cleaned['user_type'].unique())

Initial shape: (683788, 45)
Columns with empty values (original): ['health', 'spc_latin', 'spc_common', 'steward', 'guards', 'sidewalk', 'problems', 'council district', 'census tract', 'bin', 'bbl']
Cleaned shape: (674115, 45)


In [4]:
#translating strings into values for analysis
# Disable SettingWithCopyWarning
pd.options.mode.chained_assignment = None

#binary analysis for column with yes/no option
binary_analysis = ['root_stone', 'root_grate', 'root_other', 'trunk_wire',
                   'trnk_light', 'trnk_other', 'brch_light', 'brch_shoe', 'brch_other']
for name in binary_analysis:
    df_cleaned.loc[df_cleaned[name] == 'No', [name]] = 0
    df_cleaned.loc[df_cleaned[name] == 'Yes', [name]] = 1

#Option for sidewalk problem
df_cleaned.loc[df_cleaned['sidewalk'].isin(['NoDamage', 'dead', 'stump']), ['sidewalk']] = 0
df_cleaned.loc[df_cleaned['sidewalk'] == 'Damage', ['sidewalk']] = 1

#Guards problem or lack of it
df_cleaned.loc[df_cleaned['guards'].isin(['None','Unsure','dead','stump']), ['guards']] = 0
df_cleaned.loc[df_cleaned['guards'] == 'Helpful', ['guards']] = 1
df_cleaned.loc[df_cleaned['guards'] == 'Harmful', ['guards']] = -1

#Stewardship- show importance to community, not sure if helpful but show how much community relates to trees
#maybe for later analysis
df_cleaned.loc[df_cleaned['steward'].isin(['None','Unsure','dead','stump']), ['steward']] = 0
df_cleaned.loc[df_cleaned['steward'] == '1or2', ['steward']] = 1.5
df_cleaned.loc[df_cleaned['steward'] == '3or4', ['steward']] = 3.5
df_cleaned.loc[df_cleaned['steward'] == '4orMore', ['steward']] = 5

#Column to be used as a prediction
df_cleaned.loc[df_cleaned['health'].isin(['Poor','dead','stump']), ['health']] = 0
df_cleaned.loc[df_cleaned['health'] == 'Fair', ['health']] = 0.5
df_cleaned.loc[df_cleaned['health'] == 'Good', ['health']] = 1

#bias analysis user_type if neeeded
df_cleaned.loc[df_cleaned['user_type'] == 'TreesCount Staff', ['user_type']] = 0
df_cleaned.loc[df_cleaned['user_type'] == 'Volunteer', ['user_type']] = 1
df_cleaned.loc[df_cleaned['user_type'] ==  'NYC Parks Staff', ['user_type']] = -1

df_cleaned.loc[df_cleaned["curb_loc"]=="OnCurb", "curb_loc"] = 1
df_cleaned.loc[df_cleaned["curb_loc"]=="OffsetFromCurb", "curb_loc"] = 0
# Assign each species as a different feature
unique_species = df_cleaned['spc_common'].unique()  # Get unique species

# Create a DataFrame with the new columns (initialized to 0)
new_columns = pd.DataFrame(0, index=df_cleaned.index, columns=unique_species)

# Concatenate the new columns to the original DataFrame all at once
df_cleaned = pd.concat([df_cleaned, new_columns], axis=1)

# Now, update the values based on your condition
for species in unique_species:
    df_cleaned.loc[df_cleaned["spc_common"] == species, species] = 1



In [5]:
import numpy as np
# Convert 'census tract' to string and remove '.0' if present
df_cleaned['census tract'] = df_cleaned['census tract'].astype(str).str.replace('.0$', '', regex=True)

# Function to modify the census tract based on its length
def modify_census_tract(census_tract):
    modified_tracts = []
    for tract in census_tract:
        parts = list(tract)
        j = len(parts)
        if j > 4:
            part = ['0'] * (j + 1)
            for l in range(j - 2):
                part[l] = parts[l]
            part[j] = parts[j -1]
            part[j-1] = parts[j - 2]
            part[j - 2] = '.'
            modified_tracts.append(''.join(part))
        else:
            modified_tracts.append(tract)
    return modified_tracts

def striging(census):
    modified_tracts = []
    for part in census:
        part_no = str(part)
        modified_tracts.append(part_no)
    return modified_tracts

# Apply the modification function to the 'census tract' column
df_cleaned['census tract'] = modify_census_tract(df_cleaned['census tract'])
df_cleaned['census tract'] = striging(df_cleaned['census tract'])
# Safely convert to float, handling potential non-numeric values
#df_cleaned['census tract'] = pd.to_numeric(df_cleaned['census tract'], errors='coerce')
# Verify the results
#df_cleaned['census tract'] = np.round(df_cleaned['census tract'], 1)
print(df_cleaned['census tract'])


# Output the modified DataFrame
# df_cleaned.to_csv('Trees_Quantified.csv', index=False)


0            739
1            973
2            449
3            449
4            165
           ...  
683783       519
683784       707
683785       201
683786    235.02
683787      1341
Name: census tract, Length: 674115, dtype: object


In [7]:
df_cleaned.to_csv('Trees_Quantified.csv', index=False)

In [8]:
import pandas as pd

# Load the census data
df_census = pd.read_csv('Demographic_New_York_census_trackt.csv', header=1)

# Step 2: Select the 2nd and 7th columns (index 1 and 6)
df_selected_columns = df_census.iloc[:, [1, 6]]

# Rename columns
df_selected_columns = df_selected_columns.rename(columns={
    'Geographic Area Name': 'census tract',
    'Percent below poverty level!!Estimate!!Population for whom poverty status is determined': 'Poverty Percent'
})

# Convert 'Poverty Percent' column to numeric, forcing errors (like '-') to NaN
df_selected_columns['Poverty Percent'] = pd.to_numeric(df_selected_columns['Poverty Percent'], errors='coerce')

# Drop rows where 'Poverty Percent' is NaN (i.e., originally '-')
df_selected_columns = df_selected_columns.dropna(subset=['Poverty Percent'])

# Display the cleaned dataframe
print(df_selected_columns)

                                        census tract  Poverty Percent
1             Census Tract 2, Bronx County, New York             20.0
2             Census Tract 4, Bronx County, New York             13.2
3            Census Tract 16, Bronx County, New York             26.3
4            Census Tract 19, Bronx County, New York             37.1
5            Census Tract 20, Bronx County, New York             53.2
...                                              ...              ...
2161  Census Tract 303.01, Richmond County, New York              8.7
2162  Census Tract 303.02, Richmond County, New York             19.0
2163  Census Tract 319.01, Richmond County, New York             39.3
2164  Census Tract 319.02, Richmond County, New York             41.2
2165     Census Tract 323, Richmond County, New York             20.5

[2125 rows x 2 columns]


In [9]:
import pandas as pd

# Modify the census tract function
def modify_census_number(census_tract):
    modified_tracts = []
    for tract in census_tract:
        parts = tract.split(',')  # Split by the comma to isolate the tract number
        tract_number = parts[0].strip()  # Get the tract number part
        tract_number_parts = tract_number.split(' ')
        number = tract_number_parts[2]
        
        
        # Check if the tract number has a decimal (e.g., 303.0
        modified_tracts.append(number)
            
    return modified_tracts

def get_borough(census_tract):
    modified_tracts = []
    for tract in census_tract:
        parts = tract.split(',')  # Split by the comma to isolate the tract number
        tract_number = parts[1].strip()  # Get the tract number part
        tract_number_parts = tract_number.split(' ')
        number = tract_number_parts[0]
        if number == 'Richmond':
            number = 'Staten Island'
        elif number == 'New':
            number ='Manhattan'
        elif number == 'Kings':
            number = 'Brooklyn'
        # Check if the tract number has a decimal (e.g., 303.0
        modified_tracts.append(number)
            
    return modified_tracts

# Assuming you have your DataFrame loaded, apply the modification
df_selected_columns['census tract'] = df_selected_columns['census tract'].astype(str)
# Create a new column with the modified census tracts
df_selected_columns['borough'] = get_borough(df_selected_columns['census tract'])

df_selected_columns['census tract'] = modify_census_number(df_selected_columns['census tract'])

# Display the resulting DataFrame
df_selected_columns.to_csv('Economic_census.csv', index=False)